In [1]:
%load_ext dotenv

In [2]:
%dotenv

In [ ]:
# necessary libraries
from langchain.memory import ConversationSummaryMemory
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from operator import itemgetter
from langchain_core.runnables import chain

# RAG setup libraries
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_text_splitters.markdown import MarkdownHeaderTextSplitter
from langchain_text_splitters.character import CharacterTextSplitter
from langchain_community.document_loaders import Docx2txtLoader, PyPDFLoader


page = PyPDFLoader('User Query.pdf')
my_document = page.load()

#header_splitter = MarkdownHeaderTextSplitter(
   # headers_to_split_on=[("#", "Course Title"), ("##", "Lecture Title")]
#)
#header_splitted_document = header_splitter.split_text(my_document[0].page_content)
#for i in range(len(header_splitted_document)):
    #header_splitted_document[i].page_content = ' '.join(header_splitted_document[i].page_content.split())

character_splitter = CharacterTextSplitter(separator=".", chunk_size=500, chunk_overlap=50)
character_splitted_documents = character_splitter.split_documents(my_document)
for i in range(len(character_splitted_documents)):
    character_splitted_documents[i].page_content = ' '.join(character_splitted_documents[i].page_content.split())

embedding = OpenAIEmbeddings(model="text-embedding-ada-002")
vector_store = Chroma.from_documents(
    embedding=embedding,
    documents=character_splitted_documents,
    persist_directory="./TCP_directory_1"
)

retriever = vector_store.as_retriever(search_type='mmr', search_kwargs={'k': 3, 'lambda_multi': 0.5})

#chatbot memory
chat_memory = ConversationSummaryMemory(llm=ChatOpenAI(), memory_key='message_log')

#prompt template for the chatbot
TEMPLATE = """
The AI should understand the question completely and only answer strictly based on the provided document context, but it should not talk about document in the answer.  
The AI should not hallucinate.
if the question does not have enough information, the AI should ask for more details.
The AI should Deliver well-structured, polished responses that enhance the overall user experience.
If the answer is not available in the context, the AI truthfully responds: "Sorry, I don't know the answer."

Current Conversation:
{message_log}

Question Context:
{context}

Human:
{question}

AI:
"""
prompt_template = PromptTemplate.from_template(template=TEMPLATE)

# ChatOpenAI instance
chat = ChatOpenAI(
    model="gpt-4",
    temperature=0,
    max_tokens=250
)

#chain combining memory and RAG
@chain
def memory_rag_chain(question):
    # Retrieve relevant documents from the vector store
    retrieved_docs = retriever.invoke(question)
    context = "\n".join([doc.page_content for doc in retrieved_docs])

    # If no relevant context is found, a "Sorry" response is returned
    if not context.strip():
        response = "Sorry, I don't know the answer."
        chat_memory.save_context(inputs={'input': question}, outputs={'output': response})
        return response

    # Combining memory and RAG for the prompt
    chain = (
        RunnablePassthrough.assign(
            message_log=RunnableLambda(chat_memory.load_memory_variables) | itemgetter("message_log"),
            context=RunnablePassthrough()  # Pass the RAG context
        )
        | prompt_template
        | chat
        | StrOutputParser()
    )

    # Invoking the chain
    response = chain.invoke({'question': question, 'context': context})

    # Saving the interaction in memory
    chat_memory.save_context(inputs={'input': question}, outputs={'output': response})

    return response

# Usage
if __name__ == "__main__":
    print("Chatbot is ready! Type 'exit' to end the conversation.")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            break
        response = memory_rag_chain.invoke(user_input)
        print(f"Taha's Bot: {response}")



C:\Users\DELL\AppData\Local\Temp\ipykernel_4716\2571472939.py:44: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  chat_memory = ConversationSummaryMemory(llm=ChatOpenAI(), memory_key='message_log')


Chatbot is ready! Type 'exit' to end the conversation.


You:  One of my colleagues forgot to clock in for one of his shifts. How do I manually add time sheet for the coworker? I have employee permission.


Taha's Bot: As an employee, you do not have the authority to add time sheets for other coworkers. You should contact your management and request that the time sheet be added manually.


You:  One of my colleagues forgot to clock in for one of his shifts. How do I manually add time sheet for the coworker? I have employee permission.


Taha's Bot: As an employee, you do not have the authority to add time sheets for other coworkers. Please contact your management and request that the time sheet be added manually.


You:  I want to edit the shifts for the whole week.


Taha's Bot: Could you please provide more details about the specific changes you want to make? It's essential to understand your requirements thoroughly before proposing a solution.


You:  While exporting reports, is there a way to add the email field in one of the columns?


Taha's Bot: Yes, you can add the email field in one of the columns while exporting reports. You can generate a custom report and include the email as one of the columns. However, please note that you'll need to manually add the emails after downloading the report.
